<a href="https://colab.research.google.com/github/entropymator/PrimoAgent/blob/main/linera_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# To read the CSV file using pandas, you would typically do:
# import pandas as pd
# import io

# Assuming your CSV file is named 'your_file_name.csv':
# df = pd.read_csv(io.StringIO(uploaded['your_file_name.csv'].decode('utf-8')))
# print(df.head())